In [2]:
import socket 
import threading
import cv2
import pickle
import numpy as np
import struct

# Configure Socket
s = socket.socket()
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

port = 2222
ip = ""

s.bind((ip,port))
print('Socket bind complete')
s.listen()
print('Socket is now listening..............')

conn,addr=s.accept()

# Data Transfer
data = b""
payload_size = struct.calcsize(">L")

while True:
    while len(data) < payload_size:
        print("Recv: {}".format(len(data)))
        data += conn.recv(4096)

    print("Done Recv: {}".format(len(data)))
    packed_msg_size = data[:payload_size]
    data = data[payload_size:]
    msg_size = struct.unpack(">L", packed_msg_size)[0]
    print("msg_size: {}".format(msg_size))
    
    while len(data) < msg_size:
        data += conn.recv(4096)
    frame_data = data[:msg_size]
    data = data[msg_size:]

    frame=pickle.loads(frame_data, fix_imports=True, encoding="bytes")
    frame = cv2.imdecode(frame, cv2.IMREAD_COLOR)
    cv2.imshow('Server',frame)
    if cv2.waitKey(1) & 0xff==ord("q"):
        cv2.destroyAllWindows()
        s.close()

OSError: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions